In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Gemma  3 small model and tokenizer
model_name = "google/gemma-3-4b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Configure quantization for 4-bit precision
bnb_config = bnb.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the model with quantization
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,  # Adjust based on your classification task
    quantization_config=bnb_config,
    device_map="auto",
)




/home/jan/miniconda3/envs/LLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
readonly attribute

In [ ]:
# Prepare model for PEFT training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Example function to preprocess dataset
def preprocess_function(examples):
    return tokenizer(
        examples["text"],  # Adjust field name based on your dataset
        truncation=True,
        padding="max_length",
        max_length=128,
    )

# Example function to compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Example training setup

dataset = load_dataset("tweet_eval", "sentiment")
id2label = {0: 'negative', 1: 'neutral', 2: 'positive'}
# Remap the labels from integers to strings
for split in dataset.keys():
    dataset[split] = dataset[split].map(
        lambda example: {"label": id2label[example["label"]]},
        desc=f"Remapping labels in {split} split"
    )
tokenized_dataset = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()
model.save_pretrained("./gemma3-finetuned-classifier")